In [1]:
print("بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ")

بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ


In [2]:
%xmode Minimal
import os,sys,warnings,time,re,gc
from IPython.display import clear_output
warnings.filterwarnings("ignore")
from typing import Literal,Optional
from pathlib import Path
import numpy as np
import polars as pl
import polars.selectors as cs
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots
import hvplot as hv
os.cpu_count()

Exception reporting mode: Minimal


12

In [3]:
hv.extension('plotly')

In [4]:
pl.Config.set_fmt_float("full")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_float_precision(2)
pl.Config.set_tbl_rows(10)

polars.config.Config

In [5]:
path_to_train = Path("Downloads/credit_risk/train")
all_train_files = os.listdir(path_to_train)
print(len(all_train_files))
all_train_files.sort()
all_files_df = (
    pl.DataFrame({
        "index":range(len(all_train_files)),
        "filename":all_train_files
    },
    schema={"index":pl.Int8,"filename":pl.Utf8}
    )
)
all_files_df

32


index,filename
i8,str
0,"""train_applprev_1_0.parquet"""
1,"""train_applprev_1_1.parquet"""
2,"""train_applprev_2.parquet"""
3,"""train_base.parquet"""
4,"""train_credit_bureau_a_1_0.parquet"""
…,…
27,"""train_static_0_1.parquet"""
28,"""train_static_cb_0.parquet"""
29,"""train_tax_registry_a_1.parquet"""


In [6]:
feature_columns = (
    pl.read_csv("Downloads/feature_definitions.csv")
    .select(
        pl.all().shrink_dtype()
    )
)
def rename_to_full(df:pl.DataFrame) -> pl.DataFrame:    
    rename_dict = {}
    for _ in df.columns:
        des_str = feature_columns.filter(pl.col("Variable") == _)["Description"].item() if _ not in ["case_id","num_group1","num_group2"] else ""
        total_str = _+"\n"+"("+des_str+")" if _ not in ["case_id","num_group1","num_group2"] else _
        rename_dict[_] = total_str
    return df.rename(rename_dict)
print(gc.collect())
feature_columns.head()

45


Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [7]:
all_files_df.filter(pl.col("filename").str.contains("base"))

index,filename
i8,str
3,"""train_base.parquet"""


In [8]:
train_base = (
    pl.read_parquet(path_to_train/all_train_files[3])
    .select(pl.all().shrink_dtype())
    .with_columns(
        pl.col("date_decision").str.to_date(),
        pl.col("date_decision").str.to_date().dt.month().alias("month")
    )
)
print(gc.collect())
display(train_base.head())
train_base.estimated_size("mb")

0


case_id,date_decision,MONTH,WEEK_NUM,target,month
i32,date,i32,i8,i8,i8
0,2019-01-03,201901,0,0,1
1,2019-01-03,201901,0,0,1
2,2019-01-04,201901,0,0,1
3,2019-01-03,201901,0,0,1
4,2019-01-04,201901,0,1,1


21.83903217315674

In [9]:
train_base.describe()

statistic,case_id,date_decision,MONTH,WEEK_NUM,target,month
str,f64,str,f64,f64,f64,f64
"""count""",1526659.00,"""1526659""",1526659.00,1526659.00,1526659.00,1526659.00
"""null_count""",0.00,"""0""",0.00,0.00,0.00,0.00
"""mean""",1286076.57,"""2019-10-16""",201936.29,40.77,0.03,6.43
"""std""",718946.59,null,44.74,23.80,0.17,3.51
"""min""",0.00,"""2019-01-01""",201901.00,0.00,0.00,1.00
"""25%""",766198.00,"""2019-06-13""",201906.00,23.00,0.00,3.00
"""50%""",1357358.00,"""2019-10-11""",201910.00,40.00,0.00,7.00
"""75%""",1739023.00,"""2020-01-26""",202001.00,55.00,0.00,9.00
"""max""",2703454.00,"""2020-10-05""",202010.00,91.00,1.00,12.00


In [10]:
train_applprev_1 = (
    pl.scan_parquet(path_to_train/all_train_files[0])
    .drop(cs.contains("date"))
)
train_applprev_2 = (
    pl.scan_parquet(path_to_train/all_train_files[1])
    .drop(cs.contains("date"))
)
train_previous_application = (
    pl.concat(items=[train_applprev_1,train_applprev_2],how="vertical",rechunk=True)
    .with_columns(
        pl.all().shrink_dtype()
    )
    .pipe(rename_to_full)
)
train_previous_application.fetch()

case_id,actualdpd_943P (Days Past Due (DPD) of previous contract (actual).),annuity_853A (Monthly annuity for previous applications.),byoccupationinc_3656910L (Applicant's income from previous applications.),cancelreason_3545846M (Application cancellation reason.),childnum_21L (Number of children in the previous application.),credacc_actualbalance_314A (Actual balance on credit account.),credacc_credlmt_575A (Credit card credit limit provided for previous applications.),credacc_maxhisbal_375A (Maximal historical balance of previous credit account),credacc_minhisbal_90A (Minimum historical balance of previous credit accounts.),credacc_status_367L (Account status of previous credit applications.),credacc_transactions_402L (Number of transactions made with the previous credit account of the applicant.),credamount_590A (Loan amount or card limit of previous applications.),credtype_587L (Credit type of previous application.),currdebt_94A (Previous application's current debt.),district_544M (District of the address used in the previous loan application.),downpmt_134A (Previous application downpayment amount.),dtlastpmt_581D (Date of last payment made by the applicant.),dtlastpmtallstes_3545839D (Date of the applicant's last payment.),education_1138M (Applicant's education level from their previous application.),employedfrom_700D (Employment start date from the previous application.),familystate_726L (Family State in previous application of applicant.),inittransactioncode_279L (Type of the initial transaction made in the previous application of the client.),isbidproduct_390L (Flag for determining if the product is a cross-sell in previous applications.),isdebitcard_527L (Previous application flag indicating if product being applied for is a debit card.),mainoccupationinc_437A (Client's main income amount in their previous application.),maxdpdtolerance_577P (Maximum DPD with tolerance (on previous application/s).),num_group1,outstandingdebt_522A (Amount of outstanding debt on the client's previous application.),pmtnum_8L (Number of payments made for the previous application.),postype_4733339M (Type of point of sale.),profession_152M (Profession of the client during their previous loan application.),rejectreason_755M (Reason for previous application rejection.),rejectreasonclient_4145042M (Reason for rejection of the client's previous application.),revolvingaccount_394A (Revolving account that was present in the applicant's previous application.),status_219L (Previous application status.),tenor_203L (Number of instalments in the previous application.)
i32,f32,f32,f32,str,f32,f32,f32,f32,f32,str,f32,f32,str,f32,str,f32,str,str,str,str,str,str,bool,bool,f32,f32,i8,f32,f32,str,str,str,str,f32,str,f32
2,0.00,640.20,null,"""a55475b1""",0.00,null,0.00,null,null,null,null,10000.00,"""CAL""",null,"""P136_108_173""",0.00,null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""",false,null,8200.00,null,0,null,24.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",24.00
2,0.00,1682.40,null,"""a55475b1""",0.00,null,0.00,null,null,null,null,16000.00,"""CAL""",null,"""P136_108_173""",0.00,null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""",false,null,8200.00,null,1,null,12.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",12.00
3,0.00,6140.00,null,"""P94_109_143""",null,null,0.00,null,null,null,null,59999.80,"""CAL""",null,"""P131_33_167""",0.00,null,null,"""P97_36_170""","""2018-05-15""","""MARRIED""","""CASH""",false,null,11000.00,null,0,null,12.00,"""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""",null,"""D""",12.00
4,0.00,2556.60,null,"""P24_27_36""",null,null,0.00,null,null,null,null,40000.00,"""CAL""",null,"""P194_82_174""",0.00,null,null,"""a55475b1""",null,null,"""CASH""",false,null,16000.00,null,0,null,24.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""T""",24.00
5,0.00,null,null,"""P85_114_140""",null,null,null,null,null,null,null

In [11]:
train_previous_application_status = (
    pl.scan_parquet(path_to_train/all_train_files[2])
    .drop(cs.contains("date"))
    .with_columns(
        pl.all().shrink_dtype()
    )
    .pipe(rename_to_full)
)
train_previous_application_status.fetch()

case_id,cacccardblochreas_147M (Card blocking reason.),conts_type_509L (Person contact type in previous application.),credacc_cards_status_52L (Card status of the previous credit account.),num_group1,num_group2
i16,str,str,str,i8,i8
2,null,"""EMPLOYMENT_PHONE""",null,1,1
2,null,"""EMPLOYMENT_PHONE""",null,0,1
2,null,"""PRIMARY_MOBILE""",null,0,0
2,null,"""PRIMARY_MOBILE""",null,1,0
3,null,"""PRIMARY_MOBILE""",null,0,1
…,…,…,…,…,…
237,null,"""PRIMARY_MOBILE""",null,5,0
237,null,"""PRIMARY_MOBILE""",null,4,0
237,null,"""PRIMARY_MOBILE""",null,7,0
